<a href="https://colab.research.google.com/github/sandeep1214/ANDROID-/blob/main/smart_campus_energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import random
from datetime import date, timedelta

# -------------------------------
# Step 1: Create SQLite database
# -------------------------------
# Using SQLite to create a lightweight relational database for
# Smart Campus Energy Usage & Sustainability
# File will be named 'smart_campus_energy.db'
conn = sqlite3.connect("smart_campus_energy.db")
cursor = conn.cursor()

# -------------------------------
# Step 2: Create tables with constraints
# -------------------------------
# We have three tables:
# 1. Buildings: stores static information about campus buildings
# 2. EnergyUsage: stores energy and water consumption per building
# 3. UsageLogs: stores daily environmental and occupancy data
# Constraints and data types are carefully chosen to satisfy:
# - Nominal, ordinal, interval, ratio requirements
# - Foreign keys and compound keys for relational integrity
cursor.executescript("""
DROP TABLE IF EXISTS UsageLogs;
DROP TABLE IF EXISTS EnergyUsage;
DROP TABLE IF EXISTS Buildings;

-- Table 1: Buildings
CREATE TABLE Buildings (
    building_id INTEGER PRIMARY KEY,  -- Unique identifier for each building
    building_name TEXT,               -- Building name (e.g., Building_1)
    building_type TEXT CHECK (
        building_type IN ('Academic', 'Residential', 'Laboratory', 'Administrative')
    ),                                 -- Nominal data: category of building
    department TEXT,                  -- Nominal data: department responsible
    building_area_sqm REAL CHECK (building_area_sqm > 0),  -- Ratio data: meaningful zero
    sustainability_rating TEXT CHECK (
        sustainability_rating IN ('Poor', 'Fair', 'Good', 'Excellent')
    )                                 -- Ordinal data: rank from Poor -> Excellent
);

-- Table 2: EnergyUsage
CREATE TABLE EnergyUsage (
    usage_id INTEGER PRIMARY KEY,      -- Unique identifier for each usage record
    building_id INTEGER,               -- Foreign key to Buildings table
    energy_source TEXT CHECK (
        energy_source IN ('Grid', 'Solar', 'Wind')
    ),                                 -- Nominal data: energy source type
    energy_consumption_kwh REAL CHECK (energy_consumption_kwh >= 0),  -- Ratio data
    water_usage_liters REAL CHECK (water_usage_liters >= 0),           -- Ratio data
    FOREIGN KEY (building_id) REFERENCES Buildings(building_id)        -- Referential integrity
);

-- Table 3: UsageLogs
CREATE TABLE UsageLogs (
    building_id INTEGER,               -- Foreign key to Buildings
    log_date TEXT,                     -- Date of log
    temperature_celsius REAL,          -- Interval data: 0°C is arbitrary
    occupancy_level TEXT CHECK (
        occupancy_level IN ('Low', 'Medium', 'High')
    ),                                 -- Ordinal data: Low < Medium < High
    air_quality_index INTEGER,         -- Interval data: AQI relative scale
    PRIMARY KEY (building_id, log_date), -- Compound key ensures one record per building per day
    FOREIGN KEY (building_id) REFERENCES Buildings(building_id)        -- Relational integrity
);
""")

# -------------------------------
# Step 3: Reference data for random generation
# -------------------------------
# Using predefined lists for nominal/ordinal data to ensure consistency
building_types = ["Academic", "Residential", "Laboratory", "Administrative"]
departments = ["Engineering", "Science", "Business", "Arts"]
ratings = ["Poor", "Fair", "Good", "Excellent"]
energy_sources = ["Grid", "Solar", "Wind"]
occupancy_levels = ["Low", "Medium", "High"]

# -------------------------------
# Step 4: Populate Buildings table (40 rows)
# -------------------------------
# - Randomly assign building type and department (nominal)
# - Assign building area (ratio) with meaningful zero
# - Assign sustainability rating (ordinal)
# - 40 buildings is enough for relational testing
for i in range(1, 41):
    cursor.execute("""
        INSERT INTO Buildings VALUES (?, ?, ?, ?, ?, ?)
    """, (
        i,
        f"Building_{i}",                # Name is unique for clarity
        random.choice(building_types),  # Nominal data
        random.choice(departments),     # Nominal data
        round(random.uniform(500, 5000), 2),  # Ratio data
        random.choice(ratings)          # Ordinal data
    ))

# -------------------------------
# Step 5: Populate EnergyUsage table (40 rows)
# -------------------------------
# - Each building gets one energy usage record
# - Energy source is nominal
# - Energy consumption and water usage are ratio
usage_id = 1
for building_id in range(1, 41):
    cursor.execute("""
        INSERT INTO EnergyUsage VALUES (?, ?, ?, ?, ?)
    """, (
        usage_id,
        building_id,                   # FK to Buildings
        random.choice(energy_sources), # Nominal data
        round(random.uniform(1000, 10000), 2),  # Ratio data
        round(random.uniform(5000, 50000), 2)   # Ratio data
    ))
    usage_id += 1

# -------------------------------
# Step 6: Populate UsageLogs table (1200+ rows)
# -------------------------------
# - Each building logs daily environmental data for 30 days
# - temperature_celsius: interval data
# - occupancy_level: ordinal data
# - air_quality_index: interval data
# - Compound key ensures no duplicate records for same building and date
start_date = date(2025, 1, 1)
for building_id in range(1, 41):
    for day in range(30):  # 40 buildings × 30 days = 1200 rows
        log_date = start_date + timedelta(days=day)
        cursor.execute("""
            INSERT INTO UsageLogs VALUES (?, ?, ?, ?, ?)
        """, (
            building_id,
            log_date.isoformat(),
            round(random.uniform(15, 30), 2),      # Interval data: temperature
            random.choice(occupancy_levels),       # Ordinal data: occupancy
            random.randint(50, 150)                # Interval data: air quality index
        ))

# -------------------------------
# Step 7: Commit and close database
# -------------------------------
conn.commit()
conn.close()

print("Database 'smart_campus_energy.db' created successfully.")


Database 'smart_campus_energy.db' created successfully.
